# 自動微分

自動微分についてちゃんとまとめる。

In [1]:
import torch


---

## `requires_grad`

PyTorchでは、`torch.Tensor`(以下tensorと呼ぶ)の演算に対して計算グラフを構築することで、自動微分をサポートしている。

tensorは`requires_grad`という`bool`型の属性を持っており、これが`True`の場合、そのtensorを用いた演算が計算グラフに追加される。`False`の場合は追加されない。微分が必要ないと分かっている場合は`False`にしてメモリを節約した方が良い。VRAMは貴重。  
`requires_grad`はデフォルトで`False`になっている。

In [2]:
x1 = torch.tensor([0.1, 0.2, 0.3])
x1.requires_grad

False

`True`にしたかったら初期化時に引数に渡す。

In [3]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x1.requires_grad

True

後から変更することもできる。

In [4]:
x1.requires_grad = False
x1.requires_grad

False

こんなメソッドもある。

In [5]:
x1.requires_grad_(True) # デフォルト: True
x1.requires_grad

True

`float`じゃないとダメ

In [6]:
try:
    x1 = torch.tensor([1, 2, 3], requires_grad=True)
except Exception as e:
    print(e)

Only Tensors of floating point and complex dtype can require gradients


演算によって作られた新たなtensorの`requires_grad`は、演算に関わったtensorの`requires_grad`に依存する。演算に関わったtensorの中に`requires_grad`が`True`のものが1つでもあれば、新たなtensorの`requires_grad`は`True`になる。

In [7]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor([0.4, 0.5, 0.6], requires_grad=False)

z1 = x1 * 2
print(z1.requires_grad) # True

z2 = x2 * 2
print(z2.requires_grad) # False

z3 = x1 * x2
print(z3.requires_grad) # True

True
False
True



---

## leaf

計算グラフの末端にあるtensorをPyTorchの世界ではleaf tensorと呼ぶ。文字通り葉をイメージすると良い。

`torch.tensor()`によって作られたtensorは必ずleafなる。`Dataloader`から得られたtensorもleafになる。なんらかのtensorを組み合わせてできたtensorはleafにならない。  
tensorがleafかどうかは`is_leaf`で確認できる。

In [8]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor([0.4, 0.5, 0.6], requires_grad=False)
z = x1 * x2

print(x1.is_leaf)
print(x2.is_leaf)
print(z1.is_leaf)

True
True
False


この属性は変更できない

In [9]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
try:
    x1.is_leaf = False
except Exception as e:
    print(e)

attribute 'is_leaf' of 'torch._C._TensorBase' objects is not writable


leafでないtensorは`requires_grad`を変更できない

In [10]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor([0.4, 0.5, 0.6], requires_grad=False)
z = x1 * x2
try:
    z.requires_grad = True
except Exception as e:
    print(e)

you can only change requires_grad flags of leaf variables.


leaf（&`requires_grad`が`True`の）tensorはインプレースの操作ができない

In [11]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
try:
    x1.add_(1)
except Exception as e:
    print(e)

a leaf Variable that requires grad is being used in an in-place operation.


`Tensor.data`をインプレースで操作するという方法もある。ただ、そもそも`Tensor.data`を使うことが非推奨とか言われているので、やめた方が良いかも。

- [The difference between torch.tensor.data and torch.tensor - autograd - PyTorch Forums](https://discuss.pytorch.org/t/the-difference-between-torch-tensor-data-and-torch-tensor/25995)

In [12]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x1.data.add_(1)
print(x1)
print(x1.is_leaf)

tensor([1.1000, 1.2000, 1.3000], requires_grad=True)
True


以下のように値を変更することはできるが、leaf tensorではなくなる。

In [13]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x1 = x1 + 1
print(x1.is_leaf)

False


leafでないtensorは、そのtensorがどのような演算によって作られたかを`grad_fn`属性に記録している。これによって微分が可能となっている。

In [14]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = x1 * 2
print(x1.grad_fn) # leafにはない
print(z.grad_fn)

None



---

## `backward()`

微分を行うメソッド。

`backward()`を呼び出すと、そのtensorから計算グラフを遡り（逆伝播）、結果を各tensorの`grad`属性に格納する。  
別の言い方をすると、そのtensorに関わった全てのtensorで偏微分を行い、結果を各tensorの`grad`属性に格納する。

In [15]:
x1 = torch.tensor(2., requires_grad=True)
x2 = torch.tensor(5., requires_grad=False)
z = x1 * x2
z.backward()
print(x1.grad)
print(x2.grad)

tensor(5.)
None


$$
x_1 = 2, x_2 = 5 \\
z = x_1x_2 \\
\frac{\partial z}{\partial x_1} = x_2 = 5 \\
$$

なので、`x1.grad`は5になる。`x2`は`requires_grad`が`False`なのでそもそも勾配を計算しない。  
また、`requires_grad`が`True`でもleafでなけれは勾配を保持しない。

In [16]:
print(z.requires_grad)
print(z.grad)

True
None


/var/folders/0y/_tn5hh9d467bqcgqcmqkdzx80000gn/T/ipykernel_66832/1609925498.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  print(z.grad)


`retain_grad()`を呼び出すとleafでなくても勾配を保持するようになる。

In [17]:
x1 = torch.tensor(2., requires_grad=True)
x2 = torch.tensor(5., requires_grad=False)
z = x1 * x2
z.retain_grad()
z.backward()
print(z.grad)

tensor(1.)


`backward()`にスカラーを渡すと、その値から逆伝播が始まる。

In [18]:
x1 = torch.tensor(2., requires_grad=True)
x2 = torch.tensor(5., requires_grad=False)
z = x1 * x2
z.retain_grad()
z.backward(torch.tensor(3.))
print(z.grad)

tensor(3.)


tensorはスカラーでなくても良い。

In [19]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
z.backward()
print(x1.grad)

tensor([2., 2., 2.])


ただし`backward()`はスカラーに対してしか使えない。

In [20]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = x1 * 2
print(z)
try:
    z.backward()
except Exception as e:
    print(e)

tensor([0.2000, 0.4000, 0.6000], grad_fn=<MulBackward0>)
grad can be implicitly created only for scalar outputs


`backward()`を呼び出すと計算グラフが破壊されるので、2回目の`backward()`はできない。

In [21]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
z.backward() # 1回目
try:
    z.backward() # 2回目
except Exception as e:
    print(e)

Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.


`retain_graph=True`を指定すると計算グラフが保持されるので、その次の`backward()`が通る。  
複数回逆伝播を行うと`grad`は加算される。

In [22]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
for _ in range(10):
    z.backward(retain_graph=True)
print(x1.grad)

tensor([20., 20., 20.])


グラフ内のtensorが変更されてもグラフは変わらないのでグラフが作られた時の値で逆伝播が行われる。

In [23]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor(2.)
z = (x1 * x2).sum()
x2 = "hoge"
z.backward()
print(x1.grad)

tensor([2., 2., 2.])


グラフ内のtensorがインプレースで変更されると微分ができなくなる。

In [24]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor(2.)
z = (x1 * x2).sum()
x2.add_(1) # in-placeでの変更
try:
    z.backward()
except Exception as e:
    print(e)

one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).



---

## `detach()`

計算グラフから切り離したtensorを作成するメソッド。  
このtensorには勾配が流れ込まなくなる。`requires_grad`も`False`になる。

In [25]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()

x1_detached = x1.detach()
z.backward()

In [26]:
print(x1)
print(x1.grad)

tensor([0.1000, 0.2000, 0.3000], requires_grad=True)
tensor([2., 2., 2.])


In [27]:
print(x1_detached)
print(x1_detached.grad)

tensor([0.1000, 0.2000, 0.3000])
None


勾配も消える。

In [28]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
z.backward()
print(x1.grad)
print(x1.detach().grad)

tensor([2., 2., 2.])
None


`detach()`の場合はグラフ内のtensorをインプレースで操作しても逆伝播が通る。

In [29]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor(2.)
z = (x1 * x2).sum()
x2.detach_()
z.backward()
print(x1.grad)

tensor([2., 2., 2.])
